In [8]:
] activate "/Users/anarres/Documents/projects/BioXP"

 Activating environment at `~/Documents/projects/BioXP/Project.toml`


Some visual sugar

In [9]:
# ] add ProgressMeter

In [10]:
using BioXP
using ProgressMeter
using Random
using JSON
# using BenchmarkTools

### Inputs

In [11]:
## Organism JSONs from ecg (JGI)
# input_dir = "data/input/rids/"
input_dir = "data/input/rids-methanogens/"

## Master file from ecg (with dgs added from `add_dgs_to_master` .py files)
rstructs_path = "data/input/rstructs/master_from_redges-og-submission.json"

## sids to maintain in minimal seed sets
fixed_sids_path = "data/input/seeds/encel_papers_2019.json"
fixed_sid_name = "Contains_KEGGID"

## User defined seeds/targets
sid_sets_dir = "data/input/sid_randomizations-methanogens/"
targets_path = "data/input/seeds/seeds.json"
tid_name = "targets_Freilich09";

### Path to write to

In [12]:
write_mdir = "data/output/minimal_seed_sets-methanogens/";

if !ispath(write_mdir)
    mkpath(write_mdir)
end

Double check number of accessible threads

In [13]:
Threads.nthreads()

10

### Function to reorder move fixed seeds to front of random seed sets

_Allows us to figure out what the least number of compounds you need to add to an existing seed set is for viability_

In [14]:
function pop_fixed_seeds_to_front(fixed_sids::Vector{String},sid_set::Vector{String})
    intersection = intersect(fixed_sids,sid_set)
    n_skip = length(intersection)
    setdiff!(sid_set,fixed_sids)
    append!(intersection,sid_set)
    intersection,n_skip
end

pop_fixed_seeds_to_front (generic function with 1 method)

### Run minimal seed set expansions

Remember here that the threading is internal to the function

In [15]:
tids = readkeyedids(targets_path)[tid_name]
fixed_sids = readkeyedids(fixed_sids_path)[fixed_sid_name]
rstructs = readmaster(rstructs_path)

for path in readdir(input_dir)
    
    org_dir = joinpath(input_dir,path)
    
    if isdir(org_dir) &  !startswith(path,".") 
        
        p = Progress(length(readdir(org_dir)),desc="$path")
        
        for (i,fname) in collect(enumerate(readdir(org_dir)))
            if endswith(fname,".json")
    #             Random.seed!(seedoffset+i)
                rids = readids(joinpath(org_dir,fname))
                sid_sets_path = joinpath(sid_sets_dir,basename(org_dir),fname)
                sid_sets = convert(Vector{IDs},JSON.parsefile(sid_sets_path))
                
                ## Move fixed seeds to front of seed sets
                new_sid_sets = Vector{Tuple{IDs,Int64}}(undef,length(sid_sets))
                for (i,sid_set) in enumerate(sid_sets)
                    new_sid_sets[i] = pop_fixed_seeds_to_front(fixed_sids,sid_set)
                end
                
                write_path = joinpath(write_mdir,basename(org_dir),splitext(basename(fname))[1])
                
                if !ispath(joinpath(write_path))
                    mkpath(joinpath(write_path))
                end
                
#                 println(fname)
#                 println(length(sid_sets),length(new_sid_sets))
#                 println(sid_sets[1])
#                 println(typeof(new_sid_sets))

                find_minimal_seed_set(rstructs,
                    rids,
                    new_sid_sets,
                    tids,
                    write_path)

                next!(p)
            end
        end
    end
end



archaea 93%|████████████████████████████████████████▉   |  ETA: 0:00:54

### Format output

_I should format this for multithreading if I rerun it, because it takes awhile_

In [16]:
simple_output_dir = write_mdir
for path1 in readdir(simple_output_dir)
    domain_dir = joinpath(simple_output_dir,path1)
    if isdir(domain_dir) &  !startswith(path1,".") 
        p = Progress(length(readdir(domain_dir)),desc="$path1")
        Threads.@threads for (i,path2) in collect(enumerate(readdir(domain_dir)))
            org_dir = joinpath(domain_dir,path2)
            if isdir(org_dir) & !startswith(path2,".") 
                formatbioxpoutput(org_dir)
            end
            next!(p)
        end
    end
end

archaea100%|████████████████████████████████████████████| Time: 0:00:05
